In [70]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark-session").getOrCreate()
from pyspark.sql.functions import col
import matplotlib.pyplot as plt

ConnectionRefusedError: [Errno 111] Connection refused

In [71]:
path = 'gs://final_project_dsa/final_result00.csv'

df1 = spark.read.csv(path, inferSchema = True, header = False)

ConnectionRefusedError: [Errno 111] Connection refused

In [59]:
df1.printSchema()

ConnectionRefusedError: [Errno 111] Connection refused

In [12]:
path = 'gs://final_project_dsa/final_result01.csv'
df2 = spark.read.csv(path, inferSchema = True, header = False)

In [13]:
path = 'gs://final_project_dsa/final_result02.csv'
df3 = spark.read.csv(path, inferSchema = True, header = False)

In [14]:
path = 'gs://final_project_dsa/final_result03.csv'
df4 = spark.read.csv(path, inferSchema = True, header = False)

In [15]:
path = 'gs://final_project_dsa/final_result04.csv'
df5 = spark.read.csv(path, inferSchema = True, header = False)

In [16]:
path = 'gs://final_project_dsa/final_result05.csv'
df6 = spark.read.csv(path, inferSchema = True, header = False)

In [17]:
chess_dataframe = df1.union(df2).union(df3).union(df4).union(df5).union(df6)


In [23]:
zeroth_column = ["Event", "White", "Black", "Result","UTCDate", "UTCTime", "WhiteElo", "BlackElo", "WhiteRatingDiff", "BlackRatingDiff", "ECO", "Opening", "TimeControl", "Termination"]

In [26]:
for index, header in enumerate(zeroth_column):
    chess_dataframe = chess_dataframe.withColumnRenamed(f"_c{index}", header)

In [27]:
chess_dataframe.printSchema()

root
 |-- Event: string (nullable = true)
 |-- White: string (nullable = true)
 |-- Black: string (nullable = true)
 |-- Result: string (nullable = true)
 |-- UTCDate: string (nullable = true)
 |-- UTCTime: string (nullable = true)
 |-- WhiteElo: string (nullable = true)
 |-- BlackElo: string (nullable = true)
 |-- WhiteRatingDiff: string (nullable = true)
 |-- BlackRatingDiff: string (nullable = true)
 |-- ECO: string (nullable = true)
 |-- Opening: string (nullable = true)
 |-- TimeControl: string (nullable = true)
 |-- Termination: string (nullable = true)



In [28]:
chess_dataframe = chess_dataframe.withColumn('WhiteElo', col('WhiteElo').cast('Int'))

In [29]:
chess_dataframe = chess_dataframe.withColumn('BlackElo', col('BlackElo').cast('Int'))
chess_dataframe = chess_dataframe.withColumn('WhiteRatingDiff', col('WhiteRatingDiff').cast('Int'))
chess_dataframe = chess_dataframe.withColumn('BlackRatingDiff', col('BlackRatingDiff').cast('Int'))

In [30]:
chess_dataframe.printSchema()

root
 |-- Event: string (nullable = true)
 |-- White: string (nullable = true)
 |-- Black: string (nullable = true)
 |-- Result: string (nullable = true)
 |-- UTCDate: string (nullable = true)
 |-- UTCTime: string (nullable = true)
 |-- WhiteElo: integer (nullable = true)
 |-- BlackElo: integer (nullable = true)
 |-- WhiteRatingDiff: integer (nullable = true)
 |-- BlackRatingDiff: integer (nullable = true)
 |-- ECO: string (nullable = true)
 |-- Opening: string (nullable = true)
 |-- TimeControl: string (nullable = true)
 |-- Termination: string (nullable = true)



In [32]:
selected_data = chess_dataframe.select("TimeControl", "Result")

def categorize_outcome(result):
    if "1-0" in result:
        return "win"
    elif "0-1" in result:
        return "loss"
    else:
        return "draw"

In [33]:
categorize_outcome_udf = spark.udf.register("categorize_outcome", categorize_outcome)


In [40]:
filtered_data = selected_data.filter(col("TimeControl").isNotNull() & col("Result").isNotNull())


In [41]:
categorized_data = filtered_data.withColumn("Outcome", categorize_outcome_udf("Result"))


In [42]:
outcome_counts = categorized_data.groupBy("TimeControl", "Outcome").count()


In [43]:
pivot_table = outcome_counts.groupBy("TimeControl").pivot("Outcome").agg({"count": "sum"})


In [72]:
pivot_table.show(3)

ConnectionRefusedError: [Errno 111] Connection refused

In [73]:
openingCounts = chess_dataframe.groupBy("Opening").count()


ConnectionRefusedError: [Errno 111] Connection refused

In [50]:
popularOpenings = openingCounts.orderBy(col("count").desc())


NameError: name 'openingCounts' is not defined